**Libraries**

In [134]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense 
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.utils import to_categorical
import tensorflow as tf
from sklearn.metrics import accuracy_score
from keras import callbacks
from keras import layers, models
from sklearn.model_selection import train_test_split

In [166]:
alex = pd.read_excel('data/lab6_train_data.xlsx', sheet_name='Alex')
kelly = pd.read_excel('data/lab6_train_data.xlsx', sheet_name='Kelly')
test = pd.read_excel('data/lab6_train_data.xlsx', sheet_name='Test')

In [136]:
import glob
from PIL import Image
import numpy as np
import pandas as pd

def load_and_flatten_images(root_dir,
                            exts=(".jpg", ".jpeg", ".png", ".bmp"),
                            target_size=(64, 64)):
    """
    1) Walks through `root_dir`, finds all files ending in `exts` (recursively).
    2) Opens each image, converts to RGB, resizes to `target_size`.
    3) Flattens the H×W×C array into a single vector of length (H*W*C).
    4) Returns: a pandas DataFrame where each row is one image, with:
         • column "filename" (the basename of the image file), and
         • columns "pixel_0", "pixel_1", …, "pixel_{(H*W*C)-1}".
    """
    all_files = []
    for ext in exts:
        all_files.extend(glob.glob(f"{root_dir}/**/*{ext}", recursive=True))

    rows = []
    for fp in all_files:
        try:
            img = Image.open(fp).convert("RGB")
        except Exception as e:
            print(f"✗ Could not open {fp}: {e}")
            continue

        if target_size is not None:
            img = img.resize(target_size, resample=Image.BILINEAR)

        arr = np.array(img)  # shape = (H, W, 3)
        flat = arr.reshape(-1)  # shape = (H*W*3,)

        # build a single row: [filename, pixel_0, pixel_1, ..., pixel_n]
        basename = fp.split("/")[-1]  # or os.path.basename(fp)
        row = [basename] + flat.tolist()
        rows.append(row)

    if len(rows) == 0:
        return pd.DataFrame()  # empty

    # build column names: first is "filename", then "pixel_0", …, "pixel_{num_pixels-1}"
    num_pixels = target_size[0] * target_size[1] * 3
    col_names = ["Image ID"] + [f"pixel_{i}" for i in range(num_pixels)]
    df = pd.DataFrame(rows, columns=col_names)
    return df


In [137]:
#Alex Images
folder_path = "Alex_Kelly_Pics/Alex"
alex_images = load_and_flatten_images(folder_path, target_size=(64, 64))
print(alex_images.shape)         # e.g., (N_images, 1 + 224*224*3)
print(alex_images.columns[:5])  # ['filename', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3']
print(alex_images.iloc[0, :10]) # show first image's name + first few pixel values

# If you really need a NumPy array for “everything but the filename”:
X = alex_images.drop(columns=["Image ID"]).to_numpy()   # shape = (N_images, 224*224*3)
filenames = alex_images["Image ID"].tolist()

(256, 12289)
Index(['Image ID', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3'], dtype='object')
Image ID    Alex-Image119.png
pixel_0                    88
pixel_1                   103
pixel_2                   118
pixel_3                    91
pixel_4                   108
pixel_5                   122
pixel_6                    94
pixel_7                   111
pixel_8                   126
Name: 0, dtype: object
(256, 12289)
Index(['Image ID', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3'], dtype='object')
Image ID    Alex-Image119.png
pixel_0                    88
pixel_1                   103
pixel_2                   118
pixel_3                    91
pixel_4                   108
pixel_5                   122
pixel_6                    94
pixel_7                   111
pixel_8                   126
Name: 0, dtype: object


In [138]:
#Kelly Images
folder_path = "Alex_Kelly_Pics/Kelly"
kelly_images = load_and_flatten_images(folder_path, target_size=(64, 64))
print(kelly_images.shape)         # e.g., (N_images, 1 + 224*224*3)
print(kelly_images.columns[:5])  # ['filename', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3']
print(kelly_images.iloc[0, :10]) # show first image's name + first few pixel values

# If you really need a NumPy array for “everything but the filename”:
X = kelly_images.drop(columns=["Image ID"]).to_numpy()   # shape = (N_images, 224*224*3)
filenames = kelly_images["Image ID"].tolist()

(229, 12289)
Index(['Image ID', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3'], dtype='object')
Image ID    Kelly-Image124.png
pixel_0                     91
pixel_1                     87
pixel_2                     81
pixel_3                     92
pixel_4                     88
pixel_5                     81
pixel_6                     89
pixel_7                     85
pixel_8                     77
Name: 0, dtype: object
(229, 12289)
Index(['Image ID', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3'], dtype='object')
Image ID    Kelly-Image124.png
pixel_0                     91
pixel_1                     87
pixel_2                     81
pixel_3                     92
pixel_4                     88
pixel_5                     81
pixel_6                     89
pixel_7                     85
pixel_8                     77
Name: 0, dtype: object


In [139]:
#TestSet Images
folder_path = "Alex_Kelly_Pics/TestSet"
test_images = load_and_flatten_images(folder_path, target_size=(64, 64))
print(test_images.shape)         # e.g., (N_images, 1 + 224*224*3)
print(test_images.columns[:5])  # ['filename', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3']
print(test_images.iloc[0, :10]) # show first image's name + first few pixel values

# If you really need a NumPy array for “everything but the filename”:
X = test_images.drop(columns=["Image ID"]).to_numpy()   # shape = (N_images, 224*224*3)
filenames = test_images["Image ID"].tolist()

(20, 12289)
Index(['Image ID', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3'], dtype='object')
Image ID    TestSetImage01.png
pixel_0                     93
pixel_1                     88
pixel_2                     76
pixel_3                     59
pixel_4                     58
pixel_5                     56
pixel_6                     62
pixel_7                     63
pixel_8                     69
Name: 0, dtype: object
(20, 12289)
Index(['Image ID', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3'], dtype='object')
Image ID    TestSetImage01.png
pixel_0                     93
pixel_1                     88
pixel_2                     76
pixel_3                     59
pixel_4                     58
pixel_5                     56
pixel_6                     62
pixel_7                     63
pixel_8                     69
Name: 0, dtype: object


In [140]:
#combine into single image
#df["filename"].str.removesuffix(".png")


In [141]:
#stack cat df
train = pd.concat([alex, kelly], ignore_index=True)

#stack images df
train_images = pd.concat([alex_images, kelly_images], ignore_index=True)

#remove .png to merge datasets
train_images["Image ID"] = train_images["Image ID"].str.removesuffix(".png")
test_images["Image ID"] = test_images["Image ID"].str.removesuffix(".png")

#merge data
train = pd.merge(train_images, train, on="Image ID")
test = pd.merge(test_images, test, on="Image ID")

**Modeling**

Approach 1: 

* Train a (convolutional) neural network to identify whether or not there is a human in the image.
* Train a (convolutional) neural network to identify whether or not there is a castle in the image.
* Train a (convolutional) neural network to identify whether the image is taken indoors or outdoors.
* Train a (convolutional) neural network to identify the landscape of the image (e.g. city, suburb, or nature/rural)
* Choose at least 10 other features (or feature categories) that you suspect might be useful for differentiating Alex and Kelly's photos, and train individual (convolutional) neural networks to identify those.

In [142]:
model = models.Sequential()

#input layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 3)))

#hidden layers
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.25))

#output layer
model.add(layers.Dense(1, activation='sigmoid'))

#summarize
model.summary()

Model: "sequential_9"
Model: "sequential_9"


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 262144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │    33,554,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤

 Total params: 33,574,081 (128.07 MB)
 Total params: 33,574,081 (128.07 MB)


 Trainable params: 33,574,081 (128.07 MB)
 Trainable params: 33,574,081 (128.07 MB)


 Non-trainable params: 0 (0.00 B)
 Non-trainable params: 0 (0.00 B)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [143]:
#compile model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


**Train a (convolutional) neural network to identify whether or not there is a human in the image.**

In [144]:
#variable selection
X = train.drop(columns=['Image ID', 'Human', 'Castle', 'Indoors or Outdoors', 'Landscape (City, Suburb, or Nature/Rural)', 'Child/Baby', 'Animal', 'Cat', 'Dog', 'Body_of_Water', 'Car', 'Bridge', 'Food', 'Tree', 'Mountain', 'Instrument', 'Drink'])
y = train['Human']

In [145]:
X = X.to_numpy().reshape(-1, 64, 64, 3)

In [146]:
# early call back
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

#fit model
history = model.fit(
    X, y,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping],
    verbose=0
)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


In [147]:
#get predictions
y_pred = model.predict(X)
#not sure what this does
y_pred = [np.argmax(i) for i in y_pred]


 1/16 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step

 3/16 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

 5/16 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

 7/16 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

 9/16 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

11/16 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

13/16 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step


In [148]:
#get accuracy score
accuracy_score(y, y_pred)

0.6206185567010309

**Train a (convolutional) neural network to identify whether or not there is a castle in the image.**

In [149]:
#variable selection
X = train.drop(columns=['Image ID', 'Human', 'Castle', 'Indoors or Outdoors', 'Landscape (City, Suburb, or Nature/Rural)', 'Child/Baby', 'Animal', 'Cat', 'Dog', 'Body_of_Water', 'Car', 'Bridge', 'Food', 'Tree', 'Mountain', 'Instrument', 'Drink'])
y = train['Castle']

In [150]:
X = X.to_numpy().reshape(-1, 64, 64, 3)

In [151]:
# early call back
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

#fit model
history = model.fit(
    X, y,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping],
    verbose=0
)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


In [152]:
#get predictions
y_pred = model.predict(X)
#not sure what this does
y_pred = [np.argmax(i) for i in y_pred]


 1/16 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step

 2/16 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step

 4/16 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step 

 5/16 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step

 6/16 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step

...

12/16 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step

13/16 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step

14/16 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step

15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step


In [153]:
#get accuracy score
accuracy_score(y, y_pred)

0.9463917525773196

**Train a (convolutional) neural network to identify whether the image is taken indoors or outdoors.**

In [154]:
#variable selection
X = train.drop(columns=['Image ID', 'Human', 'Castle', 'Indoors or Outdoors', 'Landscape (City, Suburb, or Nature/Rural)', 'Child/Baby', 'Animal', 'Cat', 'Dog', 'Body_of_Water', 'Car', 'Bridge', 'Food', 'Tree', 'Mountain', 'Instrument', 'Drink'])
y = train['Indoors or Outdoors']

In [155]:
X = X.to_numpy().reshape(-1, 64, 64, 3)

In [156]:
# early call back
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

#fit model
history = model.fit(
    X, y,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping],
    verbose=0
)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


In [157]:
#get predictions
y_pred = model.predict(X)
#not sure what this does
y_pred = [np.argmax(i) for i in y_pred]


 1/16 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step

 3/16 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step 

 5/16 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

 7/16 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

 9/16 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

11/16 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

13/16 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step


In [158]:
#get accuracy score
accuracy_score(y, y_pred)

0.797938144329897

**Train a (convolutional) neural network to identify the landscape of the image (e.g. city, suburb, or nature/rural)**

In [159]:
model = models.Sequential()

#input layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)))

#hidden layers
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.25))

#output layer
model.add(layers.Dense(3, activation='softmax'))

#summarize
model.summary()

Model: "sequential_10"
Model: "sequential_10"


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_20 (Conv2D)              │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 128)            │     6,422,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤

 Total params: 6,441,859 (24.57 MB)
 Total params: 6,441,859 (24.57 MB)


 Trainable params: 6,441,859 (24.57 MB)
 Trainable params: 6,441,859 (24.57 MB)


 Non-trainable params: 0 (0.00 B)
 Non-trainable params: 0 (0.00 B)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [160]:
#compile model
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [165]:
#variable selection
X = train.drop(columns=['Image ID', 'Human', 'Castle', 'Indoors or Outdoors', 'Landscape (City, Suburb, or Nature/Rural)', 'Child/Baby', 'Animal', 'Cat', 'Dog', 'Body_of_Water', 'Car', 'Bridge', 'Food', 'Tree', 'Mountain', 'Instrument', 'Drink'])
y = train['Landscape (City, Suburb, or Nature/Rural)']

In [162]:
X = X.to_numpy().reshape(-1, 64, 64, 3)

In [163]:
# early call back
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

#fit model
history = model.fit(
    X, y,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping],
    verbose=0
)


ValueError: Invalid dtype: object

In [ ]:
#get predictions
y_pred = model.predict(X)
#not sure what this does
y_pred = [np.argmax(i) for i in y_pred]


In [164]:
#get accuracy score
accuracy_score(y, y_pred)

TypeError: '<' not supported between instances of 'float' and 'str'

**Choose at least 10 other features (or feature categories) that you suspect might be useful for differentiating Alex and Kelly's photos, and train individual (convolutional) neural networks to identify those.**